<a href="https://colab.research.google.com/github/NAMan00018/Final-project-mine/blob/main/comment_toxicity_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets download -d julian3833/jigsaw-toxic-comment-classification-challenge

 99% 53.0M/53.4M [00:03<00:00, 17.8MB/s]
100% 53.4M/53.4M [00:03<00:00, 15.2MB/s]


In [6]:
import zipfile
zip_ref= zipfile.ZipFile('/content/jigsaw-toxic-comment-classification-challenge.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [7]:
df=pd.read_csv('/content/train.csv')

In [8]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [8]:
# To see any comment individually
df.iloc[2]['comment_text']

"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."

In [9]:
# to see what are the opinions abt a particular cmt
df[df.columns[2:]].iloc[4]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 4, dtype: int64

  **  #### PREPROCESS COMMENTS ###**



In [10]:
from tensorflow.keras.layers import TextVectorization

In [11]:
X=df['comment_text']
Y=df[df.columns[2:]].values

In [12]:
MAX_FEATURES=300000

In [13]:
vectorizer= TextVectorization(
   max_tokens=MAX_FEATURES,
   output_sequence_length=1800,
   output_mode='int'
)

In [14]:
vectorizer.adapt(X.values)

In [17]:
vectorizer('my name is Naman')[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([    29,    109,      9, 166821])>

In [15]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [17]:
vectorized_text=vectorizer(X.values)

In [18]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[   645,     76,      2, ...,      0,      0,      0],
       [219427,     54,   2489, ...,      0,      0,      0],
       [   425,    441,     70, ...,      0,      0,      0],
       ...,
       [ 32445,   7392,    383, ...,      0,      0,      0],
       [     5,     12,    534, ...,      0,      0,      0],
       [     5,      8,    130, ...,      0,      0,      0]])>

In [19]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,Y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [20]:
batch_X, batch_Y =dataset.as_numpy_iterator().next()

In [21]:
train = dataset.take(int(len(dataset)*.7))  #70%
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) # skip 70% take 20%
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))  # skip 90% take 10%

In [22]:
train_X=train.as_numpy_iterator().next()

## ***`Build deep nueral network `***

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding

In [24]:
model=Sequential()
model.add(Embedding(MAX_FEATURES+1,32))

model.add(Bidirectional(LSTM(32,activation='tanh')))

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(6,activation='sigmoid'))

In [25]:
model.compile(optimizer='adam',loss='BinaryCrossentropy')

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          9600032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [29]:
history = model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 853s 122ms/step - loss: 0.0633 - val_loss: 0.0489


**MAKE PREDICTIONS**

In [36]:
input_text=vectorizer(' i am gonna kill u")

In [37]:
res = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 53ms/step


In [54]:
 model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 57ms/step


array([[0.9842242 , 0.17780086, 0.8829779 , 0.04737438, 0.7497161 ,
        0.11226679]], dtype=float32)

In [52]:
batch=test.as_numpy_iterator().next()

In [48]:
batch_X,batch_Y=test.as_numpy_iterator().next()

In [50]:
model.predict((batch_X)>0.5).astype(int)

1/1 [==============================] - 0s 61ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [53]:
batch_Y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [57]:
res=model.predict(batch_X)

1/1 [==============================] - 0s 61ms/step


In [59]:
res.flatten()

array([5.64083084e-03, 3.06717510e-07, 3.43519350e-04, 4.92674990e-05,
       3.61654675e-04, 6.11513824e-05, 3.90195847e-03, 1.25705569e-07,
       2.15325199e-04, 2.73740698e-05, 2.19679016e-04, 3.33866337e-05,
       2.41502374e-03, 4.03483646e-08, 1.18562661e-04, 1.29289065e-05,
       1.14232789e-04, 1.52978955e-05, 2.88233580e-03, 6.31520365e-08,
       1.49133266e-04, 1.73849785e-05, 1.46528066e-04, 2.06641180e-05,
       5.32657981e-01, 6.87633874e-03, 1.58685714e-01, 1.20573174e-02,
       1.96933910e-01, 2.84521896e-02, 1.91743486e-02, 4.90582443e-06,
       1.56122446e-03, 2.94669298e-04, 1.85880542e-03, 4.14246024e-04,
       2.70898286e-02, 1.13421811e-05, 2.36234046e-03, 5.01814880e-04,
       2.94016255e-03, 7.26629805e-04, 1.36683673e-01, 2.94668163e-04,
       1.49671789e-02, 3.17558111e-03, 2.22166125e-02, 6.17062347e-03,
       6.92304398e-04, 1.86037175e-09, 2.36970791e-05, 1.73400554e-06,
       2.04550306e-05, 1.93072174e-06, 1.59952119e-02, 3.41261625e-06,
      

**EVALUATE MODEL**

In [56]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [61]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [62]:
for batch in test.as_numpy_iterator():
  X_true,Y_true=batch

  yhat=model.predict(X_true)

  Y_true=Y_true.flatten()
  yhat=yhat.flatten()

  pre.update_state(Y_true,yhat)
  re.update_state(Y_true,yhat)
  acc.update_state(Y_true,yhat)

1/1 [==============================] - 0s 60ms/step


In [66]:
print(f'precision: {pre.result().numpy()}, recall: {re.result().numpy()},Accuracy:{acc.result().numpy()}')

precision: 0.8930648565292358, recall: 0.5686609745025635,Accuracy:0.476429283618927


**Test and Gradio**

In [130]:
!pip install gradio jinja2

In [131]:
!pip install typing_extensions

In [ ]:
pip install gradio==3.14.0

In [133]:
import gradio as gr

In [134]:
model.save('toxicity.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [135]:
model=tf.keras.models.load_model('toxicity.h5') # to be used otside the server

In [136]:
input_str=vectorizer(' i hate you will kill u soon')

In [137]:
res=model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 1s 775ms/step


In [138]:
res

array([[0.8226325 , 0.02724996, 0.45137843, 0.02073334, 0.41700992,
        0.04544419]], dtype=float32)

In [139]:
def score_comment(comment):
  vectorized_comment=vectorizer([comment])
  result=model.predict(vectorized_comment)

  text=''
  for idx,col in enumerate(df.columns[2:]):
    text += '{}:{}\n'.format(col,result[0][idx]>0.5)

  return text

In [140]:
interface = gr.Interface(fn=score_comment,
                       inputs=gr.inputs.Textbox(lines=2,placeholder='comment to score'),
                       outputs='text')


In [142]:
# prompt:

interface.launch()


IMPORTANT: You are using gradio version 2.0.0, however version 3.14.0 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Interface loading below...


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?